<a href="https://colab.research.google.com/github/Salma-Talat-Shaheen/rag-agent-langgraph/blob/main/%F0%9F%A7%A0_Model_3%2C_Section_3_Homework_Building_a_%22Self_Correcting%22_RAG_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Model 3, Section 3: Homework
> ## Building a "Self-Correcting" RAG Agent

> **🎯 Today’s Goal**: Combine all the concepts from Section 3. You will use **LangChain** and **LangGraph** to build a complete, end-to-end RAG agent that can:
> 1.  **Retrieve** documents.
> 2.  **Grade** the documents for relevance.
> 3.  **Make a decision**: either generate an answer or rewrite the query.
> 4.  **Loop** back and try again if the first attempt fails.

---

### 🤔 Recap: LangChain vs. LangGraph

* **LangChain (LCEL) 🔗**: This is our "plumbing" kit. We use it to create simple, powerful chains, like our `retriever` (from FAISS) and our `llm` (from DeepSeek). We use the pipe operator (`|`) to connect them.

* **LangGraph 📈**: This is our "agent brain" or "flowchart." We use it to connect our LangChain components with **nodes** (workers) and **conditional edges** (decision-makers). This is what allows our agent to check its work and run in loops.



Your task is to build this entire graph from scratch.

---

### 💻 Let's Get Started: Setup

First, we need to install all the libraries for LangChain, LangGraph, our models, and our vector store.

In [ ]:
# Code Cell 1: Setup
# We need langchain, langgraph, and tools to load docs and models
print("Installing necessary packages...")
# -U means upgrade, -q means quiet (less output)
!pip install langchain langgraph langchain-community langchain-huggingface faiss-cpu beautifulsoup4 bitsandbytes -U -q
print("✅ Packages installed!")

# We often need to restart the runtime *after* installing bitsandbytes
# This code handles that automatically
import os
os.kill(os.getpid(), 9)

Installing necessary packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [7]:
!pip install requests==2.32.4 --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
  Attempting uninstall: charset_normalizer
    Found existing installation: charset-normalizer 3.4.4
    Uninstalling charset-normalizer-3.4.4:
      Successfully uninstalled charset-normalizer-3.4.4
  Attempting uninstall: certifi
    Found existing installation: certifi 2025.10.5
    Uninstalling certifi

**(!!!) IMPORTANT: After the cell above runs, the runtime will restart. You must wait for it to reconnect, then run the cells *below* this point.**

### 📚 Task 1: Load Data and Build the Retriever

Your first job is to create the agent's "long-term memory."

**Your Goal:**
1.  Load the State of the Union text from the URL: `https://raw.githubusercontent.com/KxSystems/kdbai-samples/main/retrieval_augmented_generation/data/state_of_the_union.txt`
2.  Split the text into chunks (1000 characters, 200 overlap).
3.  Load the `all-MiniLM-L6-v2` embedding model.
4.  Create a `FAISS` vector store from the chunks.
5.  Define the `retriever` object (use `.as_retriever()`).

In [3]:
# Code Cell 2: Load Data and Build the Retriever
# Import all our libraries
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("Loading and preparing knowledge base...")

# 1. Load Documents
# TODO: Use WebBaseLoader to load the document from the URL
loader = WebBaseLoader(
    web_paths=("https://raw.githubusercontent.com/KxSystems/kdbai-samples/main/retrieval_augmented_generation/data/state_of_the_union.txt",),
)
docs = loader.load()

# 2. Split Documents into Chunks
# TODO: Create a RecursiveCharacterTextSplitter and split the docs
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 3. Create Embedding Model
# TODO: Load the "all-MiniLM-L6-v2" embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 4. Create Vector Store (the Retriever)
# TODO: Create a FAISS vector store from the 'splits' and 'embedding_model'
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding_model)

# 5. Define the retriever
# TODO: Create the retriever from the vectorstore
retriever = vectorstore.as_retriever()

print("✅ Knowledge base is ready.")
print(f"   -> Split into {len(splits)} chunks.")

Loading and preparing knowledge base...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Knowledge base is ready.
   -> Split into 49 chunks.


### 🤖 Task 2: Load the LLM and Define the Graph State

Now, let's load our "Generator" model and define the agent's "memory" (the `GraphState`).

**Your Goal:**
1.  Define the 4-bit `BitsAndBytesConfig` for quantization.
2.  Load the `deepseek-coder-1.3b-instruct` model and tokenizer, applying the 4-bit config.
3.  Wrap the model in a LangChain `HuggingFacePipeline`.
4.  Define the `GraphState` `TypedDict`. It must include keys for:
    * `question` (str)
    * `documents` (List[str])
    * `generation` (str)
    * `question_history` (List[str])

In [4]:
# Code Cell 3: Load the LLM and Define the Graph State
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
from typing import List, Dict
from typing_extensions import TypedDict

print("Loading Generator LLM...")

# 1. Define 4-bit config
# TODO: Create the BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 2. Load model and tokenizer
model_id = "deepseek-ai/deepseek-coder-1.3b-instruct"
# TODO: Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
# TODO: Load the model with the quantization_config
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# 3. Create the pipeline
# TODO: Create the text-generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512
)

# TODO: Wrap the pipeline in HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

print(f"✅ Generator LLM ({model_id}) loaded.")

# 4. Define the Graph State
# TODO: Define the GraphState TypedDict
class GraphState(TypedDict):
    """Represents the state of our graph.
    Attributes:
         question: The user's question
         documents: The list of retrieved documents
         generation: The LLM's final answer
         question_history: A list of all questions asked, to prevent loops
    """
    question: str
    documents: List[str]
    generation: str
    question_history: List[str]

print("✅ GraphState defined.")

Loading Generator LLM...


Device set to use cpu


✅ Generator LLM (deepseek-ai/deepseek-coder-1.3b-instruct) loaded.
✅ GraphState defined.


### 🛠️ Task 3: Define the Graph Nodes

This is the core of your agent. You need to create the four "worker" functions.

**Your Goal:**
1.  **`retrieve_node(state)`**: This function should take the `state`, retrieve documents using your `retriever`, and return a dictionary with the updated `documents` list.
2.  **`grade_documents_node(state)`**: This is the "quality checker."
    * It should loop through each document in `state["documents"]`.
    * It should use the `llm` with a simple **"yes/no" prompt** to decide if the document is relevant to the `question`.
    * It should return a dictionary with the `documents` list *updated* to contain **only** the relevant documents.
3.  **`generate_node(state)`**: This is the "answerer."
    * It should take the `question` and the *filtered* `documents` from the state.
    * It should use a prompt to ask the `llm` to generate a final answer.
    * It should return a dictionary with the final `generation`.
4.  **`rewrite_node(state)`**: This is the "re-do" worker.
    * It should check for loops (if the current `question` is already in `question_history`).
    * If not a loop, it should add the question to the history.
    * It should use the `llm` with a prompt to rewrite the `question`.
    * It should return a dictionary with the *new* `question` and the updated `question_history`.

In [5]:
# Code Cell 4: Define the Graph Nodes
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

print("Defining graph nodes...")

# --- NODE 1: RETRIEVE ---
# TODO: Define the retrieve_node function
def retrieve_node(state):
    """Takes the question and retrieves documents."""
    print("---NODE: RETRIEVE---")
    question = state["question"]
    documents = retriever.invoke(question)
    doc_texts = [doc.page_content for doc in documents]

    return {
        "documents": doc_texts,
        "question": question,
        "question_history": state.get("question_history", [])
    }

# --- NODE 2: GRADE DOCUMENTS ---
# TODO: Define the grade_documents_node function
def grade_documents_node(state):
    """Checks if the retrieved documents are relevant to the question."""
    print("---NODE: GRADE DOCUMENTS---")
    question = state["question"]
    documents = state["documents"]

    # Prompt for the grader
    prompt_template = """You are a grader. Your job is to check if a
    retrieved document is relevant to a user question.
    Respond with a *single word*: 'yes' if relevant, 'no' if not.

    Document: {document}
    Question: {question}

    Answer:"""
    prompt = PromptTemplate.from_template(prompt_template)
    grader_chain = prompt | llm | StrOutputParser()

    relevant_docs = []
    for doc in documents:
        try:
            result = grader_chain.invoke({"question": question, "document": doc})
            score = result.strip().lower()
            if "yes" in score:
                print("  -> Grader Decision: Relevant")
                relevant_docs.append(doc)
            else:
                print("  -> Grader Decision: NOT Relevant")
        except Exception as e:
            print(f"  -> Grader: Error - {e}")
            continue

    return {
        "documents": relevant_docs,
        "question": question,
        "question_history": state.get("question_history", [])
    }

# --- NODE 3: GENERATE ---
# TODO: Define the generate_node function
def generate_node(state):
    """Takes the question and documents, and generates an answer."""
    print("---NODE: GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG prompt
    prompt_template = """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Be concise.

    Question: {question}
    Context: {context}

    Helpful Answer:"""
    prompt = PromptTemplate.from_template(prompt_template)
    rag_chain = prompt | llm | StrOutputParser()

    generation = rag_chain.invoke({"context": "\n".join(documents), "question": question})

    return {
        "documents": documents,
        "question": question,
        "generation": generation,
        "question_history": state.get("question_history", [])
    }

# --- NODE 4: REWRITE ---
# TODO: Define the rewrite_node function
def rewrite_node(state):
    """Takes the question and rewrites it to be a better search query, with loop prevention."""
    print("---NODE: REWRITE QUERY---")
    question = state["question"]
    question_history = state.get("question_history", [])

    # Loop Prevention
    if question in question_history:
        print("  -> Detected loop: Ending.")
        return {
            "documents": [],
            "generation": "Could not find relevant information after multiple attempts.",
            "question_history": question_history
        }

    question_history.append(question)

    # Rewrite prompt
    prompt_template = """You are a query rewriter. Rewrite the following question to be
    a concise and specific search query for a vector database.
    Respond ONLY with the rewritten query, nothing else.

    Original Question: {question}

    Rewritten Query:"""
    prompt = PromptTemplate.from_template(prompt_template)
    rewrite_chain = prompt | llm | StrOutputParser()

    new_question = rewrite_chain.invoke({"question": question}).strip()
    print(f"   -> Rewritten question: {new_question}")

    return {
        "question": new_question,
        "question_history": question_history,
        "documents": []
    }

print("✅ All nodes defined.")

Defining graph nodes...
✅ All nodes defined.


### 🕸️ Task 4: Define the Conditional Edges (The Brain)

Now you need to define the agent's decision-making logic.

**Your Goal:**
1.  Create a function `decide_to_generate_or_rewrite(state)`.
2.  This function must check the `state` and return a string:
    * If a `generation` was already created (by the loop-prevention in `rewrite_node`), return `"end"`.
    * If there are relevant `documents` in the state, return `"generate"`.
    * If there are no relevant documents, check the `question_history` length. If it's `> 2`, return `"end"` (to give up).
    * Otherwise, return `"rewrite"`.

In [6]:
# Code Cell 5: Define the Conditional Edges (The Brain)
print("Defining conditional logic...")

# TODO: Define the conditional edge function
def decide_to_generate_or_rewrite(state):
    """Decides whether to generate an answer, rewrite the query, or end."""
    print("---CONDITIONAL EDGE---")
    documents = state["documents"]
    question_history = state.get("question_history", [])

    # Check for loop prevention
    if state.get("generation"):
        print("  -> Decision: Loop detected. END.")
        return "end"

    # If we have relevant docs, generate
    if len(documents) > 0:
        print("  -> Decision: Relevant documents found. GENERATE.")
        return "generate"
    else:
        # If no docs, check rewrite limit
        if len(question_history) > 2: # 1 original + 2 rewrites
            print("  -> Decision: Max rewrites reached. END.")
            return "end" # Give up
        else:
            print("  -> Decision: No relevant documents. REWRITE.")
            return "rewrite"

print("✅ Conditional logic defined.")

Defining conditional logic...
✅ Conditional logic defined.


### ⚙️ Task 5: Build and Compile the Graph

You have all the pieces! Now, wire them all together.

**Your Goal:**
1.  Create a `StateGraph(GraphState)`.
2.  Add all four of your nodes: `retrieve`, `grade_documents`, `generate`, and `rewrite`.
3.  Set the `retrieve` node as the **entry point**.
4.  Add the **edges**:
    * `retrieve` -> `grade_documents`
    * `rewrite` -> `retrieve` (This creates the loop!)
    * `generate` -> `END`
5.  Add the **conditional edge** starting from `grade_documents` and using your `decide_to_generate_or_rewrite` function.
6.  `compile()` the graph into a runnable `app`.

In [7]:
# Code Cell 6: Build and Compile the Graph
from langgraph.graph import END, StateGraph

print("Building the graph...")

# TODO: Create the StateGraph
workflow = StateGraph(GraphState)

# TODO: Add all 4 nodes
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("grade_documents", grade_documents_node)
workflow.add_node("generate", generate_node)
workflow.add_node("rewrite", rewrite_node)

# TODO: Set the entry point
workflow.set_entry_point("retrieve")

# TODO: Add the standard edges
workflow.add_edge("retrieve", "grade_documents")
workflow.add_edge("rewrite", "retrieve") # The loop
workflow.add_edge("generate", END)

# TODO: Add the conditional edge
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate_or_rewrite,
    {
        "generate": "generate",
        "rewrite": "rewrite",
        "end": END
    }
)

# TODO: Compile the graph
app = workflow.compile()
print("✅ Graph compiled successfully!")

Building the graph...
✅ Graph compiled successfully!


### 🚀 Task 6: Run Your Agent!

It's time to test your creation.

**Your Goal:**
1.  Define a `query` (e.g., "What did the President say about the PRO Act?").
2.  Define the initial `inputs` dictionary (don't forget `question_history: []`).
3.  Run `app.stream(inputs)` and loop through the outputs to see your agent work.
4.  Print the final answer!

In [9]:
# Code Cell 7: Run Your Agent!

print("--- 🚀 Test 1: Running RAG Agent (Good Query) ---")
# TODO: Define a query and initial inputs
query = "What did the President say about the PRO Act?"
inputs = {"question": query, "documents": [], "generation": "", "question_history": []}

# TODO: Run app.stream() and print the final 'generate' output
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"\n--- Output from Node: {key} ---")
        if key == "generate":
            print(f"   -> 🤖 **Final Answer:** {value['generation']}")

print("\n--- 🏁 Run Complete ---")

--- 🚀 Test 1: Running RAG Agent (Good Query) ---

---NODE: RETRIEVE---
  
--- Output from Node: retrieve ---
---NODE: GRADE DOCUMENTS---
  -> Grader Decision: Relevant
  -> Grader Decision: Relevant
  -> Grader Decision: Relevant
  -> Grader Decision: NOT Relevant 
 
--- Output from Node: grade_documents ---
---CONDITIONAL EDGE---
  -> Decision: Relevant documents found. GENERATE.

--- Output from Node: grade_documents ---
---NODE: GENERATE---

--- Output from Node: generate ---
   -> 🤖 **Final Answer:** The President urged the Senate to pass the **PRO Act** to ensure the right of workers to organize a union, stating that the law would make it easier for workers to join a union and collectively bargain for better wages. (The specific text of the answer may vary slightly based on the LLM's output.)

--- Output from Node: __end__ ---

--- 🏁 Run Complete ---


### 🧪 Self-Assessment

Run this final cell to validate your agent's logic. This will test its ability to answer a question that requires a **rewrite**.

In [10]:
# Code Cell 8: Self-Assessment
# This cell will test your agent's ability to handle a query
# that requires rewriting.

print("--- 🧪 Self-Assessment: Testing Rewrite Logic ---")

# This query is vague and should be rewritten
query = "What about infrastructure?"
inputs = {"question": query, "documents": [], "generation": "", "question_history": []}

final_answer = ""
rewrite_detected = False

try:
    for output in app.stream(inputs):
        for key, value in output.items():
            print(f"\n--- Output from Node: {key} ---")
            if key == "rewrite":
                rewrite_detected = True
                print("   ->  detected a rewrite!")
            if key == "generate":
                final_answer = value.get("generation", "")
            elif key == "__end__" and not final_answer:
                final_answer = value.get("generation", "Agent ended (likely via rewrite loop prevention).")

    print("\n--- 🏁 Assessment Complete ---")
    print(f"   -> Original Query: '{query}'")
    print(f"   -> Final Answer: {final_answer}")

    if rewrite_detected:
        print("✅  Success! The agent correctly identified the bad context and triggered a rewrite.")
    else:
        print("❌  Failed. The agent did not trigger the 'rewrite' node. Check your 'grade_documents' node and conditional logic.")

    if "sorry" in final_answer.lower() or "don't know" in final_answer.lower():
         print("⚠️  Warning: The agent tried but couldn't find a good answer (this might be ok!).")
    elif final_answer:
         print("✅  Success! The agent successfully generated an answer after its process.")
    else:
         print("❌  Failed. The agent did not produce a final answer.")

except Exception as e:
    print(f"❌ Test failed with an error: {e}")

--- 🧪 Self-Assessment: Testing Rewrite Logic ---

---NODE: RETRIEVE---
  
--- Output from Node: retrieve ---
---NODE: GRADE DOCUMENTS---
  -> Grader Decision: NOT Relevant
  -> Grader Decision: NOT Relevant
  -> Grader Decision: NOT Relevant
  -> Grader Decision: NOT Relevant
 
--- Output from Node: grade_documents ---
---CONDITIONAL EDGE---
  -> Decision: No relevant documents. REWRITE.

--- Output from Node: grade_documents ---
---NODE: REWRITE QUERY---
  -> Rewritten question: President's plan for improving infrastructure
  ->  detected a rewrite!

--- Output from Node: rewrite ---
---NODE: RETRIEVE---
  
--- Output from Node: retrieve ---
---NODE: GRADE DOCUMENTS---
  -> Grader Decision: Relevant
  -> Grader Decision: Relevant
  -> Grader Decision: Relevant
  -> Grader Decision: Relevant
 
--- Output from Node: grade_documents ---
---CONDITIONAL EDGE---
  -> Decision: Relevant documents found. GENERATE.

--- Output from Node: grade_documents ---
---NODE: GENERATE---

--- Output fro

### 🚀 Share Your Success & Submit Your Work

You've just built a complex, self-correcting AI agent! This is a major milestone.

**1. Official Submission (Google Classroom)**
* **What:** A shareable link to your completed Google Colab notebook.
* **How:**
    1.  In your Colab notebook, click the **"Share"** button (top right).
    2.  Under "General access," change it from "Restricted" to **"Anyone with the link"** can **"View"**.
    3.  Click **"Copy link"**.
* **Where:** Submit this link to the **"Week 3 Homework"** assignment in Google Classroom.
* **Why:** This is how you get credit for completing the section.

---

**2. Optional: Build Your Professional Portfolio (Highly Recommended)**
This is the *perfect* project to show off.

* **LinkedIn Post:** Write a post explaining how you built an agent that can "think." Mention **LangGraph**, **RAG**, and **conditional logic**. Show a screenshot of your agent successfully rewriting a query!
* **YouTube Video:** Record a 2-minute video. Run your agent with the "vague query" from the self-assessment and *explain* how it decides to rewrite the question. This is impressive stuff.

Good luck!